In [54]:
import sys
import operator
import requests
import json
import twitter
import config as cfg
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# get configuration strings from config file
api_key = cfg.api_key
api_secret = cfg.api_secret
bearer = cfg.bearer
token = cfg.access_token
token_secret = cfg.access_secret

In [55]:
# create Twitter object
api = twitter.Api(consumer_key = api_key,
                 consumer_secret = api_secret,
                 access_token_key = token,
                 access_token_secret = token_secret)

In [92]:
tweets = api.GetSearch(term = '$nio',count = 25,lang = 'en')

In [93]:
# To create the dataframe, I will have to create a list, iterate through the tweets, and append to that list.

# Define the columns
cols = ['text']
# Create empty list to append
lst = []

for tweet in tweets:
   lst.append([tweet.text])

# Create the dataframe using lst and cols
df_main = pd.DataFrame(lst,columns = cols)


In [94]:
df_main

,text
0,*NIO PLANS STOCK SALE WORTH $3.12 BILLION AS C...
1,$NIO\n\nADDING SUB $45 AS MUCH AS POSSIBE; htt...
2,"Rome wasn't built in a day, this is a long ho..."
3,RT @traderstewie: Our old friend $NIO setting ...
4,RT @EngineeringRobo: $NIO \n\nCongratulations ...
5,$penn $dkng $aapl $hexo $ino $mara $aal $tsla ...
6,RT @EliteOptions2: 🎉 3 Month Holiday Giveaway ...
7,Get : $5K Bonus\nspreads as low as \n0.0 Pips\...
8,Get now\n$5000\ndeposit bonus\n\noffers link ...
9,Watchlist for tomorrow: 📈\n1. $NIO\n2. $RKT\n3...


In [95]:
# Convert to list for VADER

lst_text = df_main.values.tolist()

# Adding VADER Sentiment and storing those results to a list to turn into a dataframe
sentimentResults = []
analyzer = SentimentIntensityAnalyzer()
for sentence in lst_text:
    vs = analyzer.polarity_scores(sentence)
    sentimentResults.append(vs)

# Convert vader reult into a dataframe by concatenating two dataframes
text = pd.DataFrame(lst_text, columns= ['Text'])
# Convert the sentiment analysis results list to a dataframe
df_sentimentResults = pd.DataFrame(sentimentResults)
dataset = pd.concat([text,df_sentimentResults],axis = 1)

In [96]:
# The compound is -1 (most extreme negative) and +1 (most extreme positive). 0 for neutral
# Drop neutral tweets for now
dropped = dataset[dataset.compound != 0]

In [97]:
dropped

,Text,neg,neu,pos,compound
0,*NIO PLANS STOCK SALE WORTH $3.12 BILLION AS C...,0.000,0.649,0.351,0.7963
2,"Rome wasn't built in a day, this is a long ho...",0.000,0.778,0.222,0.7184
3,RT @traderstewie: Our old friend $NIO setting ...,0.000,0.641,0.359,0.8807
4,RT @EngineeringRobo: $NIO \n\nCongratulations ...,0.000,0.639,0.361,0.8941
6,RT @EliteOptions2: 🎉 3 Month Holiday Giveaway ...,0.000,0.795,0.205,0.6892
7,Get : $5K Bonus\nspreads as low as \n0.0 Pips\...,0.085,0.772,0.142,0.3400
8,Get now\n$5000\ndeposit bonus\n\noffers link ...,0.000,0.821,0.179,0.5423
11,RT @InvestorWisdom: $NIO chart looks bottomed ...,0.000,0.809,0.191,0.7096
12,RT @traderstewie: Our old friend $NIO setting ...,0.000,0.641,0.359,0.8807
15,$NIO $XPEV $LI 👏🏻12/15/20 166th battery swap s...,0.000,0.786,0.214,0.4588
